In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
from io import BytesIO
import datetime
from urllib import request,error

In [2]:
def yahoo_download(url):
	url = "https://finance.yahoo.com/quote/" + url + "/history"
	head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
	main_driver = webdriver.Chrome('/Users/lou_tun_chieh/Desktop/webdriver/chromedriver')  # 注意你們放CHROMEDRIVER的位置
	#告訴chromedriver 等下要找的element 如果沒有找到，要等10秒讓他們生完
	main_driver.implicitly_wait(10)
	main_driver.get(url)
	cookie_list = main_driver.get_cookies()
	cookies_dict = {}
	for cookie in cookie_list:
		cookies_dict['B'] = cookie['value']
	#print(cookies_dict)

	date_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/span/input')
	date_btn_ele.click()
	main_driver.implicitly_wait(10)

	#點取date button
	date_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/input[1]').clear()
	date_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/input[1]').send_keys("01/01/2016")
	#date_btn_ele.click()
	main_driver.implicitly_wait(10)

	##點取done button
	done_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/div[3]/button[1]')
	done_btn_ele.click()
	main_driver.implicitly_wait(10)

	#點取apply button
	done_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button')
	done_btn_ele.click()
	main_driver.implicitly_wait(10)
	time.sleep(30)
	
	#得到歷史資料的csv檔
	export_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a')
	csv_url = export_btn_ele.get_attribute('href')
	main_driver.quit()
	return csv_url, cookies_dict

In [3]:
def write_down_csv(etf_name):
	
	tmp_url, tmp_cookie  = yahoo_download(etf_name)
	download = requests.get(tmp_url, cookies=tmp_cookie)
	df = pd.read_csv(BytesIO(download.content))
	#將該變數轉成pandas的dataframe格式
	df = pd.DataFrame(df)
	df = df.rename(columns={'Adj Close': etf_name})
	"""
	filename = etf_name + ".csv"
	with open (filename, 'wb') as handle:
		for block in download.iter_content(1024):
			handle.write(block)
	"""
	return df



In [4]:
head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
#必須帶有瀏覽器等資訊
etf_list_1 = "../Municipal Bond ETF List (29).csv"
etf_list_2 = "../Target Maturity Date Corporate Bond ETF List (24).csv"

df_1 = pd.read_csv(etf_list_1)
df_2 = pd.read_csv(etf_list_2)
df = pd.concat([df_1, df_2])

df = df[df["Inception"] < "2016"]
df = df.reset_index(drop=True)

tmp_df =  write_down_csv(df["Symbol"][0])
#Date = tmp_df["Date"]
result =  pd.concat([tmp_df["Date"], tmp_df[df["Symbol"][0]]], axis=1)
for i in range(1, len(df["Symbol"])):
	tmp_df =  write_down_csv(df["Symbol"][i])
	#print(tmp_df[df["Symbol"][i]])
	result =  pd.concat([result, tmp_df[df["Symbol"][i]]], axis=1)
	#print(result)
result

,Date,MUB,VTEB,SHM,TFI,SUB,ITM,PZA,FMB,MUNI,...,BSCN,IBDO,IBDP,BSCO,IBDQ,BSCP,IBDC,IBCD,IBDD,IBCE
0,2015-12-31,103.068077,47.981476,46.976612,45.477581,102.533638,44.715878,23.068096,47.914940,49.956993,...,18.184570,22.139408,21.846327,17.956078,21.728523,17.890175,23.465050,22.357132,22.777237,20.984543
1,2016-01-04,102.844643,48.037949,46.880180,45.365639,102.427200,44.734516,23.113382,47.850441,49.864429,...,18.257561,22.226423,21.800821,18.028734,21.801043,17.981270,23.426466,22.246979,22.529444,20.949131
2,2016-01-05,102.965645,48.113224,46.938038,45.608162,102.465912,44.921059,23.167725,48.043919,49.938473,...,18.312311,22.125668,21.826303,18.028734,21.737593,18.072355,23.480932,22.361725,22.593033,21.048742
3,2016-01-06,103.421837,48.310833,47.053745,45.757385,102.649765,45.144924,23.258287,48.246601,50.160625,...,18.239311,22.217264,21.964664,18.136810,21.782913,18.108789,23.512705,22.409918,22.711452,21.152771
4,2016-01-07,103.570786,48.282593,47.073032,45.831989,102.688484,45.238205,23.276403,48.263191,50.243942,...,18.294058,22.221842,21.937361,18.127728,21.810099,18.061436,23.537670,22.471876,22.654438,21.185982
5,2016-01-08,103.617363,48.320240,47.015175,45.906620,102.756218,45.163582,23.276403,48.117619,50.401299,...,18.357937,22.258480,21.955561,18.155890,21.837301,18.154337,23.558088,22.506298,22.750927,21.216970
6,2016-01-11,103.449783,48.188515,47.053745,45.794689,102.649765,45.088959,23.213011,48.265034,50.151371,...,18.284933,22.224588,21.928255,18.119556,21.755718,18.092398,23.592133,22.492533,22.540409,21.157194
7,2016-01-12,103.487000,48.160271,47.111599,45.831989,102.678795,45.163582,23.249235,48.320309,50.197662,...,18.348804,22.194368,21.957386,18.183134,21.828232,18.163456,23.537670,22.501711,22.658823,21.343138
8,2016-01-13,103.533569,48.207325,47.073032,45.794689,102.717529,45.144924,23.249235,48.136047,50.179142,...,18.312311,22.230999,21.991974,18.183134,21.879002,18.183493,23.542204,22.508600,22.698294,21.265661
9,2016-01-14,103.347366,48.169670,47.092308,45.776031,102.727188,45.126270,23.267345,48.255806,50.327240,...,18.339684,22.203526,21.964664,18.164967,21.900763,18.113352,23.569437,22.517784,22.792591,21.276733
